In [1]:
%load_ext autoreload
%autoreload
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

from datasets.kitti_raw_monosf import KITTI_Raw_KittiSplit_Train, KITTI_Raw_KittiSplit_Valid
from models.model_monosceneflow import MonoSceneFlow
from augmentations import Augmentation_SceneFlow

In [2]:
# args
args = {
    'batch_size' : 1,
    'num_workers' : 2,
    'epochs': 1, 
    'lr': 2e-4,
    'momentum': 0.9,
    'beta': 0.999,
    'weight_decay': 0.0,
    'train': True
}

In [6]:
data_root = '/external/datasets/kitti_data_jpg/'
train_dataset = KITTI_Raw_KittiSplit_Train(args={}, root=data_root)
train_loader = DataLoader(train_dataset, batch_size=args['batch_size'], num_workers=args['num_workers'], shuffle=True)
val_dataset = KITTI_Raw_KittiSplit_Valid(args={}, root=data_root)
val_loader = DataLoader(val_dataset, batch_size=args['batch_size'], num_workers=args['num_workers'], shuffle=False)

In [4]:
%autoreload 

augmentation = Augmentation_SceneFlow(args)
model = MonoSceneFlow(args)

params = model.parameters()

optimizer = Adam(params, lr=args['lr'], betas=[args['momentum'], args['beta']], weight_decay=args['weight_decay'])

In [15]:
torch.cuda.set_device(0)

%autoreload
for data in train_loader:
    model(augmentation(data))
    break

RuntimeError: CUDA error: an illegal memory access was encountered

In [55]:
def train_one_epoch(model, dataloader, optimizer, augmentations):

  model.train()

  for i, data in enumerate(dataloader):
    continue

  return

In [58]:
for epoch in range(args['epochs']):
    loss_one_epoch = train_one_epoch(model, dataloader, optimizer, augmentations=augmentation)
    optimizer.zero_grad()
    loss_one_epoch.backward()
    

NameError: name 'optimizer' is not defined